# QC-Py-06 : Options Trading dans QuantConnect

**Duree estimee** : 75 minutes

**Objectifs d'apprentissage** :
1. Comprendre les concepts fondamentaux des options (Call, Put, Strike, Expiration)
2. Ajouter et configurer des options dans QuantConnect
3. Naviguer dans l'OptionChain pour selectionner des contrats
4. Utiliser les Greeks pour la gestion du risque
5. Implementer des strategies classiques (Covered Call, Protective Put, Spreads, Iron Condor)

**Prerequis** :
- Notebooks QC-Py-01 a QC-Py-04 completes
- Comprehension de base des marches financiers
- Notions de Python intermediaire

**Note importante** : Les donnees d'options sont disponibles gratuitement sur QuantConnect (free tier) pour les equities US comme SPY.

---
## Plan du notebook

1. **Introduction aux Options** (10 min)
2. **Ajouter des Options dans QuantConnect** (15 min)
3. **Options Chain : Navigation et Selection** (20 min)
4. **Les Greeks** (15 min)
5. **Strategies de Base** (30 min)
   - Covered Call
   - Protective Put
   - Vertical Spread
6. **Iron Condor** (15 min)
7. **Exercise et Assignment** (10 min)
8. **Strategie Complete : Covered Call sur SPY** (20 min)
9. **Conclusion**

---
## 1. Introduction aux Options (10 min)

### Qu'est-ce qu'une option ?

Une **option** est un contrat derivatif qui donne a son detenteur le **droit** (mais pas l'obligation) d'acheter ou de vendre un actif sous-jacent a un prix predetermine, avant ou a une date d'expiration.

### Les deux types d'options

| Type | Definition | Acheteur | Vendeur |
|------|------------|----------|----------|
| **CALL** | Droit d'**acheter** le sous-jacent | Paye la prime, espere hausse | Recoit la prime, risque hausse |
| **PUT** | Droit de **vendre** le sous-jacent | Paye la prime, espere baisse | Recoit la prime, risque baisse |

### Terminologie essentielle

| Terme | Definition | Exemple (SPY a 450$) |
|-------|------------|----------------------|
| **Strike** | Prix d'exercice | Strike 460$ |
| **Expiration** | Date limite du contrat | 15 mars 2024 |
| **Premium** | Prix paye/recu pour l'option | 5.50$ par action |
| **Underlying** | Actif sous-jacent | SPY (ETF S&P 500) |
| **Contract Size** | 1 contrat = 100 actions | 1 contrat = 100 actions SPY |

### ITM / ATM / OTM

Ces termes decrivent la relation entre le strike et le prix actuel du sous-jacent :

| Terme | Call | Put | Signification |
|-------|------|-----|---------------|
| **ITM** (In-The-Money) | Strike < Prix | Strike > Prix | Option a de la valeur intrinseque |
| **ATM** (At-The-Money) | Strike = Prix | Strike = Prix | Strike proche du prix actuel |
| **OTM** (Out-of-The-Money) | Strike > Prix | Strike < Prix | Option n'a que de la valeur temporelle |

**Exemple** : SPY se trade a 450$
- Call strike 440$ : **ITM** (peut acheter a 440$ un actif valant 450$)
- Call strike 450$ : **ATM** (strike = prix actuel)
- Call strike 460$ : **OTM** (peut acheter a 460$ un actif valant 450$, pas interessant)

### Pourquoi trader des options ?

| Objectif | Strategie | Exemple |
|----------|-----------|----------|
| **Speculation** | Acheter calls/puts | Pari sur hausse/baisse avec effet de levier |
| **Couverture (Hedging)** | Protective Put | Proteger un portefeuille contre la baisse |
| **Revenu** | Covered Call | Generer du premium sur des actions detenues |
| **Volatilite** | Straddle, Iron Condor | Profiter de marches calmes ou volatils |

### Diagramme des payoffs

```
LONG CALL (Achat d'un Call)              LONG PUT (Achat d'un Put)

Profit                                    Profit
  ^                                         ^
  |           /                             |  \                     
  |          /                              |   \                    
  |         /                               |    \                   
--|--------/--------> Prix                  --|----\-------> Prix    
  |  -P   K                                 |      K   -P            
  |                                         |                        
                                                                     
K = Strike                                  K = Strike               
P = Premium paye                            P = Premium paye         
Gain max = Illimite                         Gain max = K - P         
Perte max = Premium                         Perte max = Premium      
```

**Point cle** : L'acheteur d'une option a un risque limite (premium paye) mais un gain potentiel eleve. Le vendeur a un gain limite (premium recu) mais un risque potentiel eleve.

---
## 2. Ajouter des Options dans QuantConnect (15 min)

### Methode de base : AddOption()

Pour trader des options sur QuantConnect, il faut d'abord les ajouter a l'univers de l'algorithme avec `AddOption()`.

```python
# Ajouter les options sur SPY
option = self.AddOption("SPY", Resolution.Minute)
```

**Important** : Les options necessitent `Resolution.Minute` au minimum. `Resolution.Daily` n'est pas supporte pour les options car les prix changent frequemment durant la journee.

### Configuration du filtre avec SetFilter()

Les options sur un actif populaire comme SPY ont des centaines de contrats disponibles (differents strikes et expirations). Pour eviter de surcharger l'algorithme, on utilise `SetFilter()` pour filtrer les contrats :

```python
option.SetFilter(minStrike, maxStrike, minExpiry, maxExpiry)
```

| Parametre | Description | Exemple |
|-----------|-------------|---------|
| `minStrike` | Strike minimum (relatif au prix) | -5 (5 strikes sous le prix actuel) |
| `maxStrike` | Strike maximum (relatif au prix) | +5 (5 strikes au-dessus) |
| `minExpiry` | Expiration minimum | `timedelta(30)` (min 30 jours) |
| `maxExpiry` | Expiration maximum | `timedelta(60)` (max 60 jours) |

In [ ]:
# Configuration de base des options sur SPY
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *
from datetime import timedelta

class BasicOptionsSetup(QCAlgorithm):
    """
    Configuration de base pour trader des options sur SPY.
    """
    
    def Initialize(self):
        # Configuration du backtest
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Ajouter l'equity sous-jacent (facultatif mais recommande)
        self.equity = self.AddEquity("SPY", Resolution.Minute)
        
        # Ajouter les options sur SPY
        self.option = self.AddOption("SPY", Resolution.Minute)
        
        # Stocker le symbol de l'option pour reference
        self.option_symbol = self.option.Symbol
        
        # Configurer le filtre d'options
        # -5/+5 strikes autour du prix actuel
        # Expiration entre 30 et 60 jours
        self.option.SetFilter(
            minStrike=-5,
            maxStrike=5,
            minExpiry=timedelta(days=30),
            maxExpiry=timedelta(days=60)
        )
        
        self.Log("Options setup completed")
        self.Log(f"Option Symbol: {self.option_symbol}")
    
    def OnData(self, slice):
        # Verifier si des options sont disponibles
        if not slice.OptionChains.ContainsKey(self.option_symbol):
            return
        
        # Obtenir la chaine d'options
        chain = slice.OptionChains[self.option_symbol]
        
        # Compter les contrats disponibles
        contracts = list(chain)
        if len(contracts) > 0:
            self.Debug(f"Found {len(contracts)} option contracts")

### Filtres avances

Vous pouvez creer des filtres plus sophistiques avec une fonction lambda :

```python
# Filtre personnalise
self.option.SetFilter(lambda universe: universe
    .Strikes(-10, +10)              # 10 strikes de chaque cote
    .Expiration(timedelta(7), timedelta(45))  # 7 a 45 jours
    .IncludeWeeklys()               # Inclure les options hebdomadaires
)
```

**Options hebdomadaires (Weeklys)** : Expirent chaque vendredi, tres populaires pour le trading a court terme.

---
## 3. Options Chain : Navigation et Selection (20 min)

### Structure de l'OptionChain

Dans `OnData`, les donnees d'options sont disponibles via `slice.OptionChains`. C'est un dictionnaire dont les cles sont les symbols d'options et les valeurs sont des `OptionChain`.

```python
def OnData(self, slice):
    # Verifier la disponibilite
    if not slice.OptionChains.ContainsKey(self.option_symbol):
        return
    
    # Obtenir la chaine
    chain = slice.OptionChains[self.option_symbol]
    
    # chain est iterable - chaque element est un OptionContract
    for contract in chain:
        # contract.Strike, contract.Expiry, contract.Right, etc.
        pass
```

### Proprietes d'un OptionContract

| Propriete | Type | Description |
|-----------|------|-------------|
| `Symbol` | Symbol | Identifiant unique du contrat |
| `Strike` | Decimal | Prix d'exercice |
| `Expiry` | DateTime | Date d'expiration |
| `Right` | OptionRight | `OptionRight.Call` ou `OptionRight.Put` |
| `UnderlyingLastPrice` | Decimal | Dernier prix du sous-jacent |
| `BidPrice` | Decimal | Prix d'achat (bid) |
| `AskPrice` | Decimal | Prix de vente (ask) |
| `LastPrice` | Decimal | Dernier prix echange |
| `OpenInterest` | int | Nombre de contrats ouverts |
| `ImpliedVolatility` | Decimal | Volatilite implicite |
| `Greeks` | Greeks | Delta, Gamma, Theta, Vega, Rho |

In [ ]:
# Navigation dans l'OptionChain
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *
from datetime import timedelta

class OptionChainNavigation(QCAlgorithm):
    """
    Demonstration de la navigation dans l'OptionChain.
    """
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 3, 31)
        self.SetCash(100000)
        
        self.equity = self.AddEquity("SPY", Resolution.Minute)
        self.option = self.AddOption("SPY", Resolution.Minute)
        self.option_symbol = self.option.Symbol
        
        self.option.SetFilter(-5, 5, timedelta(30), timedelta(60))
        
        # Flag pour n'afficher qu'une fois
        self.logged = False
    
    def OnData(self, slice):
        if self.logged:
            return
        
        if not slice.OptionChains.ContainsKey(self.option_symbol):
            return
        
        chain = slice.OptionChains[self.option_symbol]
        contracts = list(chain)
        
        if len(contracts) == 0:
            return
        
        # Prix actuel du sous-jacent
        underlying_price = chain.Underlying.Price
        self.Log(f"\n=== Option Chain Analysis ===")
        self.Log(f"SPY Price: ${underlying_price:.2f}")
        self.Log(f"Total contracts: {len(contracts)}")
        
        # Separer calls et puts
        calls = [c for c in chain if c.Right == OptionRight.Call]
        puts = [c for c in chain if c.Right == OptionRight.Put]
        
        self.Log(f"Calls: {len(calls)} | Puts: {len(puts)}")
        
        # Trouver le call ATM (strike le plus proche du prix)
        if calls:
            atm_call = min(calls, key=lambda x: abs(x.Strike - underlying_price))
            self.Log(f"\nATM Call: Strike={atm_call.Strike}, Expiry={atm_call.Expiry.date()}")
            self.Log(f"  Bid: ${atm_call.BidPrice:.2f} | Ask: ${atm_call.AskPrice:.2f}")
            self.Log(f"  IV: {atm_call.ImpliedVolatility:.2%} | OI: {atm_call.OpenInterest}")
        
        # Trouver le put ATM
        if puts:
            atm_put = min(puts, key=lambda x: abs(x.Strike - underlying_price))
            self.Log(f"\nATM Put: Strike={atm_put.Strike}, Expiry={atm_put.Expiry.date()}")
            self.Log(f"  Bid: ${atm_put.BidPrice:.2f} | Ask: ${atm_put.AskPrice:.2f}")
            self.Log(f"  IV: {atm_put.ImpliedVolatility:.2%} | OI: {atm_put.OpenInterest}")
        
        self.logged = True

### Selection de contrats par criteres

Pour selectionner des contrats specifiques, utilisez des list comprehensions avec des filtres :

In [ ]:
# Exemples de selection de contrats
# A copier dans l'IDE QuantConnect

def select_contracts(self, chain, underlying_price):
    """
    Exemples de selection de contrats d'options.
    """
    
    # 1. Tous les calls
    calls = [c for c in chain if c.Right == OptionRight.Call]
    
    # 2. Tous les puts
    puts = [c for c in chain if c.Right == OptionRight.Put]
    
    # 3. Call ATM (strike le plus proche du prix)
    atm_call = min(calls, key=lambda x: abs(x.Strike - underlying_price)) if calls else None
    
    # 4. Put ATM
    atm_put = min(puts, key=lambda x: abs(x.Strike - underlying_price)) if puts else None
    
    # 5. Calls OTM (strike > prix)
    otm_calls = [c for c in calls if c.Strike > underlying_price]
    
    # 6. Puts OTM (strike < prix)
    otm_puts = [c for c in puts if c.Strike < underlying_price]
    
    # 7. Calls ITM (strike < prix)
    itm_calls = [c for c in calls if c.Strike < underlying_price]
    
    # 8. Contrats expirant dans exactement X jours
    target_expiry = self.Time + timedelta(days=45)
    contracts_45d = [c for c in chain 
                     if abs((c.Expiry - target_expiry).days) <= 3]  # tolerance 3 jours
    
    # 9. Contrats avec volume suffisant (OpenInterest > 100)
    liquid_contracts = [c for c in chain if c.OpenInterest > 100]
    
    # 10. Call OTM le plus proche (pour covered call)
    if otm_calls:
        nearest_otm_call = min(otm_calls, key=lambda x: x.Strike)
    
    # 11. Selection combinee: Call OTM, expiration 30-45 jours, delta ~0.30
    ideal_calls = [c for c in otm_calls 
                   if 30 <= (c.Expiry - self.Time).days <= 45
                   and 0.25 <= c.Greeks.Delta <= 0.35]
    
    return {
        'atm_call': atm_call,
        'atm_put': atm_put,
        'otm_calls': otm_calls,
        'otm_puts': otm_puts,
        'ideal_calls': ideal_calls
    }

### Bonnes pratiques de selection

| Critere | Pourquoi | Valeur recommandee |
|---------|----------|--------------------|
| **Open Interest** | Liquidite, facilite d'execution | > 100 pour options liquides |
| **Bid-Ask Spread** | Cout de transaction | < 5% du prix de l'option |
| **Expiration** | Decay temporel, flexibilite | 30-60 jours pour strategies de revenu |
| **Strike** | Probabilite d'exercice | OTM 5-10% pour covered calls |

---
## 4. Les Greeks (15 min)

### Introduction aux Greeks

Les **Greeks** sont des mesures de sensibilite qui quantifient comment le prix d'une option change en fonction de differents facteurs. Ils sont essentiels pour la gestion du risque.

### Les 5 Greeks principaux

| Greek | Mesure | Interpretation | Valeur typique |
|-------|--------|----------------|----------------|
| **Delta** | Sensibilite au prix du sous-jacent | +1$ sous-jacent = +Delta$ option | Call: 0 a 1, Put: -1 a 0 |
| **Gamma** | Acceleration du delta | Mesure la courbure | Eleve pres ATM |
| **Theta** | Decay temporel (par jour) | Perte de valeur avec le temps | Toujours negatif (pour acheteur) |
| **Vega** | Sensibilite a la volatilite | +1% IV = +Vega$ option | Eleve pour options longues |
| **Rho** | Sensibilite aux taux d'interet | +1% taux = +Rho$ option | Faible impact a court terme |

### Delta en detail

Le **Delta** est le Greek le plus utilise. Il represente :

1. **Sensibilite au prix** : Si delta = 0.50, une hausse de 1$ du sous-jacent augmente l'option de 0.50$
2. **Probabilite approximative** : Delta ~= probabilite que l'option expire ITM
3. **Equivalent actions** : Un call avec delta 0.50 se comporte comme 50 actions

```
Delta selon le strike:

Delta
  1.0 |  ______
      |        \
  0.5 |         \____  <- ATM (delta ~0.50)
      |               \
  0.0 |________________\
      ITM    ATM    OTM
```

### Theta - Le temps joue contre l'acheteur

Le **Theta** represente la perte de valeur quotidienne due au passage du temps. C'est le principal ennemi de l'acheteur d'options.

```
Valeur de l'option vs Temps restant:

Valeur
  ^                      | Decay accelere
  |  \                   | proche de
  |    \                 | l'expiration
  |      \               v
  |        \__
  |            \___
  |                 \____
  +------------------------> Temps
  90j    60j    30j    0j
```

**Conseil** : Les vendeurs d'options profitent du theta decay, les acheteurs le subissent.

In [ ]:
# Acces aux Greeks dans QuantConnect
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *
from datetime import timedelta

class GreeksAnalysis(QCAlgorithm):
    """
    Analyse des Greeks pour la gestion du risque.
    """
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 3, 31)
        self.SetCash(100000)
        
        self.equity = self.AddEquity("SPY", Resolution.Minute)
        self.option = self.AddOption("SPY", Resolution.Minute)
        self.option_symbol = self.option.Symbol
        
        self.option.SetFilter(-3, 3, timedelta(30), timedelta(45))
        
        self.logged = False
    
    def OnData(self, slice):
        if self.logged:
            return
        
        if not slice.OptionChains.ContainsKey(self.option_symbol):
            return
        
        chain = slice.OptionChains[self.option_symbol]
        underlying_price = chain.Underlying.Price
        
        # Selectionner le call ATM
        calls = [c for c in chain if c.Right == OptionRight.Call]
        if not calls:
            return
        
        atm_call = min(calls, key=lambda x: abs(x.Strike - underlying_price))
        
        # Afficher les Greeks
        self.Log(f"\n=== Greeks Analysis for ATM Call ===")
        self.Log(f"Underlying: SPY @ ${underlying_price:.2f}")
        self.Log(f"Contract: Strike={atm_call.Strike}, Expiry={atm_call.Expiry.date()}")
        self.Log(f"Price: ${atm_call.LastPrice:.2f}")
        self.Log(f"")
        
        # Acces aux Greeks via contract.Greeks
        greeks = atm_call.Greeks
        
        self.Log(f"DELTA: {greeks.Delta:.4f}")
        self.Log(f"  -> Si SPY +$1, option +${greeks.Delta:.4f}")
        self.Log(f"  -> Probabilite ITM: ~{greeks.Delta*100:.1f}%")
        self.Log(f"")
        
        self.Log(f"GAMMA: {greeks.Gamma:.4f}")
        self.Log(f"  -> Si SPY +$1, delta change de {greeks.Gamma:.4f}")
        self.Log(f"")
        
        self.Log(f"THETA: {greeks.Theta:.4f}")
        self.Log(f"  -> Perte quotidienne: ${abs(greeks.Theta):.4f}")
        self.Log(f"")
        
        self.Log(f"VEGA: {greeks.Vega:.4f}")
        self.Log(f"  -> Si IV +1%, option +${greeks.Vega:.4f}")
        self.Log(f"")
        
        self.Log(f"RHO: {greeks.Rho:.4f}")
        self.Log(f"  -> Si taux +1%, option +${greeks.Rho:.4f}")
        
        self.logged = True
    
    def SelectByDelta(self, calls, target_delta=0.30, tolerance=0.05):
        """
        Selectionner un call par son delta.
        Utile pour les covered calls (delta 0.25-0.35 recommande).
        """
        matching = [c for c in calls 
                    if abs(c.Greeks.Delta - target_delta) <= tolerance]
        
        if matching:
            # Prendre celui le plus proche du delta cible
            return min(matching, key=lambda x: abs(x.Greeks.Delta - target_delta))
        return None

### Utilisation des Greeks pour le Risk Management

| Objectif | Greek a surveiller | Action |
|----------|-------------------|--------|
| Limiter l'exposition au prix | Delta | Maintenir delta total proche de 0 (delta neutral) |
| Reduire le decay temporel | Theta | Vendre des options (theta positif) |
| Profiter de la volatilite | Vega | Acheter options si IV attendue en hausse |
| Position conservatrice | Delta | Choisir des options avec delta < 0.30 |

---
## 5. Strategies de Base (30 min)

### Vue d'ensemble des strategies

| Strategie | Composition | Objectif | Risque max | Gain max |
|-----------|-------------|----------|-----------|----------|
| **Covered Call** | Long stock + Short call OTM | Revenu (premium) | Stock -> 0 | Premium + (Strike - Achat) |
| **Protective Put** | Long stock + Long put | Protection baisse | Premium paye | Illimite |
| **Bull Call Spread** | Long call + Short call (strike+) | Hausse limitee | Premium net | Diff strikes - Premium |
| **Bear Put Spread** | Long put + Short put (strike-) | Baisse limitee | Premium net | Diff strikes - Premium |

### 5a. Covered Call (10 min)

#### Principe

Le **Covered Call** est la strategie d'options la plus populaire pour generer du revenu. Elle consiste a :

1. **Detenir** 100 actions du sous-jacent (ou un multiple de 100)
2. **Vendre** un call OTM (strike au-dessus du prix actuel)

```
COVERED CALL = Long 100 shares + Short 1 Call OTM

Payoff:
         ___________
        /
       /
------/-------------> Prix
     Strike

Gain max = Premium + (Strike - Prix d'achat)
Perte max = Prix d'achat - Premium (si action -> 0)
```

#### Avantages et inconvenients

| Avantages | Inconvenients |
|-----------|---------------|
| Genere du revenu regulier | Limite le gain potentiel |
| Reduit le cout d'achat (breakeven) | Oblige a vendre si assigne |
| Strategie simple a executer | Necessite de detenir l'action |

#### Quand utiliser

- Vous etes **neutre a moderement haussier** sur l'action
- Vous acceptez de **vendre a un prix cible** (strike)
- Vous voulez **reduire la volatilite** de votre portefeuille

In [ ]:
# Implementation d'un Covered Call
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *
from datetime import timedelta

class CoveredCallStrategy(QCAlgorithm):
    """
    Strategie Covered Call sur SPY.
    
    1. Acheter 100 actions SPY
    2. Vendre un call OTM (delta ~0.30)
    3. Laisser expirer ou roller avant expiration
    """
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(50000)  # Capital suffisant pour 100 SPY
        
        # Ajouter l'equity et les options
        self.equity = self.AddEquity("SPY", Resolution.Minute)
        self.spy = self.equity.Symbol
        
        self.option = self.AddOption("SPY", Resolution.Minute)
        self.option_symbol = self.option.Symbol
        
        # Filtre: calls OTM, expiration 30-45 jours
        self.option.SetFilter(
            minStrike=0,   # ATM et au-dessus
            maxStrike=10,  # 10 strikes OTM max
            minExpiry=timedelta(days=30),
            maxExpiry=timedelta(days=45)
        )
        
        # Tracking
        self.current_call = None
        self.premium_collected = 0
        
        self.Log("Covered Call Strategy initialized")
    
    def OnData(self, slice):
        # Etape 1: Acheter 100 actions si pas deja fait
        if not self.Portfolio[self.spy].Invested:
            self.MarketOrder(self.spy, 100)
            self.Log(f"Bought 100 SPY shares")
            return  # Attendre le prochain tick pour vendre le call
        
        # Etape 2: Verifier si on a deja un call vendu
        if self.current_call is not None:
            # Verifier si le call a expire
            if self.Time >= self.current_call.ID.Date:
                self.Log(f"Call expired: {self.current_call}")
                self.current_call = None
            else:
                return  # On a deja un call, attendre
        
        # Etape 3: Vendre un nouveau call OTM
        if not slice.OptionChains.ContainsKey(self.option_symbol):
            return
        
        chain = slice.OptionChains[self.option_symbol]
        underlying_price = chain.Underlying.Price
        
        # Filtrer les calls OTM
        calls = [c for c in chain 
                 if c.Right == OptionRight.Call 
                 and c.Strike > underlying_price]
        
        if not calls:
            return
        
        # Selectionner le call avec delta proche de 0.30
        target_delta = 0.30
        selected_call = min(calls, 
                           key=lambda x: abs(x.Greeks.Delta - target_delta))
        
        # Verifier que le delta est raisonnable
        if selected_call.Greeks.Delta < 0.20 or selected_call.Greeks.Delta > 0.40:
            self.Debug(f"No suitable call found (delta={selected_call.Greeks.Delta:.2f})")
            return
        
        # Vendre le call (quantity negative = vente)
        self.MarketOrder(selected_call.Symbol, -1)
        self.current_call = selected_call.Symbol
        
        # Calculer le premium (approximation avec mid price)
        premium = (selected_call.BidPrice + selected_call.AskPrice) / 2 * 100
        self.premium_collected += premium
        
        self.Log(f"Sold Call: Strike={selected_call.Strike}, " +
                 f"Expiry={selected_call.Expiry.date()}, " +
                 f"Delta={selected_call.Greeks.Delta:.2f}, " +
                 f"Premium=${premium:.2f}")
    
    def OnEndOfAlgorithm(self):
        self.Log(f"\n=== Covered Call Summary ===")
        self.Log(f"Total Premium Collected: ${self.premium_collected:.2f}")
        self.Log(f"Final Portfolio Value: ${self.Portfolio.TotalPortfolioValue:.2f}")

### 5b. Protective Put (10 min)

#### Principe

Le **Protective Put** est une strategie de couverture (hedging). Elle consiste a :

1. **Detenir** 100 actions du sous-jacent
2. **Acheter** un put (generalement ATM ou legerement OTM)

```
PROTECTIVE PUT = Long 100 shares + Long 1 Put

Payoff:
          /
         /
        /
-------/____________ <- Protection en cas de baisse
      Strike

Gain max = Illimite (hausse de l'action)
Perte max = (Prix achat - Strike) + Premium
```

#### Analogie avec l'assurance

Le Protective Put est souvent compare a une **police d'assurance** :

| Assurance auto | Protective Put |
|----------------|----------------|
| Prime mensuelle | Premium du put |
| Franchise | Difference (Prix - Strike) |
| Indemnisation si accident | Protection si chute du cours |
| Expire apres 1 an | Expire a l'echeance |

In [ ]:
# Implementation d'un Protective Put
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *
from datetime import timedelta

class ProtectivePutStrategy(QCAlgorithm):
    """
    Strategie Protective Put sur SPY.
    
    Acheter 100 actions + un put ATM pour se proteger.
    """
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 6, 30)
        self.SetCash(60000)
        
        self.equity = self.AddEquity("SPY", Resolution.Minute)
        self.spy = self.equity.Symbol
        
        self.option = self.AddOption("SPY", Resolution.Minute)
        self.option_symbol = self.option.Symbol
        
        # Filtre: puts ATM et OTM, expiration 30-60 jours
        self.option.SetFilter(
            minStrike=-5,  # 5 strikes sous ATM
            maxStrike=0,   # jusqu'a ATM
            minExpiry=timedelta(days=30),
            maxExpiry=timedelta(days=60)
        )
        
        self.current_put = None
        self.protection_cost = 0
        
        self.Log("Protective Put Strategy initialized")
    
    def OnData(self, slice):
        # Etape 1: Acheter 100 actions
        if not self.Portfolio[self.spy].Invested:
            self.MarketOrder(self.spy, 100)
            self.Log(f"Bought 100 SPY shares")
            return
        
        # Etape 2: Verifier si on a deja un put
        if self.current_put is not None:
            if self.Time >= self.current_put.ID.Date:
                self.Log(f"Put expired: {self.current_put}")
                self.current_put = None
            else:
                return
        
        # Etape 3: Acheter un put de protection
        if not slice.OptionChains.ContainsKey(self.option_symbol):
            return
        
        chain = slice.OptionChains[self.option_symbol]
        underlying_price = chain.Underlying.Price
        
        # Filtrer les puts
        puts = [c for c in chain if c.Right == OptionRight.Put]
        
        if not puts:
            return
        
        # Selectionner le put ATM (strike le plus proche du prix)
        atm_put = min(puts, key=lambda x: abs(x.Strike - underlying_price))
        
        # Acheter le put
        self.MarketOrder(atm_put.Symbol, 1)
        self.current_put = atm_put.Symbol
        
        # Cout de la protection
        cost = (atm_put.AskPrice) * 100
        self.protection_cost += cost
        
        self.Log(f"Bought Put: Strike={atm_put.Strike}, " +
                 f"Expiry={atm_put.Expiry.date()}, " +
                 f"Cost=${cost:.2f}")
        self.Log(f"Protected from losses below ${atm_put.Strike}")
    
    def OnEndOfAlgorithm(self):
        self.Log(f"\n=== Protective Put Summary ===")
        self.Log(f"Total Protection Cost: ${self.protection_cost:.2f}")
        self.Log(f"Final Portfolio Value: ${self.Portfolio.TotalPortfolioValue:.2f}")

### 5c. Vertical Spread (10 min)

#### Bull Call Spread

Un **Bull Call Spread** est une strategie haussiere a risque et gain limites :

1. **Acheter** un call (strike bas, plus cher)
2. **Vendre** un call (strike plus haut, moins cher)

```
BULL CALL SPREAD = Long Call (K1) + Short Call (K2, K2 > K1)

Payoff:
           _________  <- Gain max = K2 - K1 - Premium net
          /
         /
--------/____________ <- Perte max = Premium net
       K1    K2
```

#### Bear Put Spread

Un **Bear Put Spread** est l'equivalent baissier :

1. **Acheter** un put (strike haut, plus cher)
2. **Vendre** un put (strike bas, moins cher)

#### Avantages des spreads

| Avantage | Explication |
|----------|-------------|
| Cout reduit | La vente finance partiellement l'achat |
| Risque limite | Perte max = premium net paye |
| Meilleur breakeven | Point mort plus proche du prix actuel |

In [ ]:
# Implementation d'un Bull Call Spread
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *
from datetime import timedelta

class BullCallSpreadStrategy(QCAlgorithm):
    """
    Strategie Bull Call Spread sur SPY.
    
    Acheter un call ATM + Vendre un call OTM.
    """
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 6, 30)
        self.SetCash(10000)
        
        self.option = self.AddOption("SPY", Resolution.Minute)
        self.option_symbol = self.option.Symbol
        
        self.option.SetFilter(-2, 5, timedelta(30), timedelta(45))
        
        self.spread_opened = False
        
        self.Log("Bull Call Spread Strategy initialized")
    
    def OnData(self, slice):
        if self.spread_opened:
            return
        
        if not slice.OptionChains.ContainsKey(self.option_symbol):
            return
        
        chain = slice.OptionChains[self.option_symbol]
        underlying_price = chain.Underlying.Price
        
        # Filtrer les calls
        calls = [c for c in chain if c.Right == OptionRight.Call]
        
        if len(calls) < 2:
            return
        
        # Trier par strike
        calls = sorted(calls, key=lambda x: x.Strike)
        
        # Trouver le call ATM (long leg)
        long_call = min(calls, key=lambda x: abs(x.Strike - underlying_price))
        
        # Trouver le call OTM (short leg) - environ 5$ au-dessus
        short_calls = [c for c in calls 
                       if c.Strike > long_call.Strike 
                       and c.Expiry == long_call.Expiry]
        
        if not short_calls:
            return
        
        # Prendre le premier call OTM
        short_call = short_calls[0]
        
        # Calculer les couts
        long_cost = long_call.AskPrice * 100
        short_credit = short_call.BidPrice * 100
        net_debit = long_cost - short_credit
        max_profit = (short_call.Strike - long_call.Strike) * 100 - net_debit
        
        self.Log(f"\n=== Bull Call Spread ===")
        self.Log(f"SPY @ ${underlying_price:.2f}")
        self.Log(f"Long Call: Strike={long_call.Strike}, Cost=${long_cost:.2f}")
        self.Log(f"Short Call: Strike={short_call.Strike}, Credit=${short_credit:.2f}")
        self.Log(f"Net Debit (Max Loss): ${net_debit:.2f}")
        self.Log(f"Max Profit: ${max_profit:.2f}")
        self.Log(f"Breakeven: ${long_call.Strike + net_debit/100:.2f}")
        
        # Executer le spread
        self.MarketOrder(long_call.Symbol, 1)   # Acheter 1 call
        self.MarketOrder(short_call.Symbol, -1) # Vendre 1 call
        
        self.spread_opened = True

---
## 6. Iron Condor (15 min)

### Principe

L'**Iron Condor** est une strategie neutre qui profite des marches lateraux (peu de mouvement). Elle combine un Bull Put Spread et un Bear Call Spread :

```
IRON CONDOR = Bull Put Spread + Bear Call Spread

Composition:
1. Sell Put (OTM, strike bas)      <- Genere du credit
2. Buy Put (plus OTM, strike+bas)  <- Limite le risque
3. Sell Call (OTM, strike haut)    <- Genere du credit
4. Buy Call (plus OTM, strike+haut) <- Limite le risque
```

### Diagramme de payoff

```
Profit
   ^            ___________  <- Max profit = Credit total
   |           /           \
   |          /             \
---+---------/---------------\---------> Prix
   |        /                 \
   |       /                   \______  <- Max loss
   |      K1   K2       K3    K4
   
K1 = Long Put strike
K2 = Short Put strike  
K3 = Short Call strike
K4 = Long Call strike

Zone de profit: Prix reste entre K2 et K3
```

### Caracteristiques

| Caracteristique | Valeur |
|-----------------|--------|
| Max Profit | Credit net recu |
| Max Loss | (Largeur du spread - Credit) |
| Breakeven bas | K2 - Credit |
| Breakeven haut | K3 + Credit |
| Probabilite de profit | ~60-70% (si strikes bien places) |

### Quand utiliser

- Marche **range-bound** (sans tendance claire)
- Volatilite implicite **elevee** (pour recevoir plus de premium)
- Vous anticipez **peu de mouvement** jusqu'a expiration

In [ ]:
# Implementation d'un Iron Condor
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *
from datetime import timedelta

class IronCondorStrategy(QCAlgorithm):
    """
    Strategie Iron Condor sur SPY.
    
    Strategie neutre pour marches lateraux.
    """
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 6, 30)
        self.SetCash(20000)
        
        self.option = self.AddOption("SPY", Resolution.Minute)
        self.option_symbol = self.option.Symbol
        
        # Filtre large pour avoir assez de strikes
        self.option.SetFilter(-10, 10, timedelta(30), timedelta(45))
        
        self.condor_opened = False
        
        self.Log("Iron Condor Strategy initialized")
    
    def OnData(self, slice):
        if self.condor_opened:
            return
        
        if not slice.OptionChains.ContainsKey(self.option_symbol):
            return
        
        chain = slice.OptionChains[self.option_symbol]
        underlying_price = chain.Underlying.Price
        
        # Separer calls et puts
        calls = sorted([c for c in chain if c.Right == OptionRight.Call], 
                       key=lambda x: x.Strike)
        puts = sorted([c for c in chain if c.Right == OptionRight.Put], 
                      key=lambda x: x.Strike)
        
        if len(calls) < 4 or len(puts) < 4:
            return
        
        # Configuration: wings a 5% du prix, spreads de 5$
        put_wing_strike = underlying_price * 0.95
        call_wing_strike = underlying_price * 1.05
        
        # Trouver les contrats
        # Put side: Buy put (K1) + Sell put (K2, K2 > K1)
        short_put = min(puts, key=lambda x: abs(x.Strike - put_wing_strike))
        long_puts = [p for p in puts if p.Strike < short_put.Strike 
                     and p.Expiry == short_put.Expiry]
        
        # Call side: Sell call (K3) + Buy call (K4, K4 > K3)
        short_call = min(calls, key=lambda x: abs(x.Strike - call_wing_strike))
        long_calls = [c for c in calls if c.Strike > short_call.Strike 
                      and c.Expiry == short_call.Expiry]
        
        if not long_puts or not long_calls:
            return
        
        long_put = long_puts[-1]  # Le plus proche du short
        long_call = long_calls[0]  # Le plus proche du short
        
        # Calculer le credit et risque
        credit_put = (short_put.BidPrice - long_put.AskPrice) * 100
        credit_call = (short_call.BidPrice - long_call.AskPrice) * 100
        total_credit = credit_put + credit_call
        
        put_spread_width = (short_put.Strike - long_put.Strike) * 100
        call_spread_width = (long_call.Strike - short_call.Strike) * 100
        max_loss = max(put_spread_width, call_spread_width) - total_credit
        
        self.Log(f"\n=== Iron Condor ===")
        self.Log(f"SPY @ ${underlying_price:.2f}")
        self.Log(f"Expiry: {short_put.Expiry.date()}")
        self.Log(f"")
        self.Log(f"PUT SIDE (Bull Put Spread):")
        self.Log(f"  Long Put: K={long_put.Strike}")
        self.Log(f"  Short Put: K={short_put.Strike}")
        self.Log(f"  Credit: ${credit_put:.2f}")
        self.Log(f"")
        self.Log(f"CALL SIDE (Bear Call Spread):")
        self.Log(f"  Short Call: K={short_call.Strike}")
        self.Log(f"  Long Call: K={long_call.Strike}")
        self.Log(f"  Credit: ${credit_call:.2f}")
        self.Log(f"")
        self.Log(f"Total Credit (Max Profit): ${total_credit:.2f}")
        self.Log(f"Max Loss: ${max_loss:.2f}")
        self.Log(f"Profit Zone: ${short_put.Strike} - ${short_call.Strike}")
        
        # Executer l'Iron Condor (4 ordres)
        self.MarketOrder(long_put.Symbol, 1)     # Buy put
        self.MarketOrder(short_put.Symbol, -1)   # Sell put
        self.MarketOrder(short_call.Symbol, -1)  # Sell call
        self.MarketOrder(long_call.Symbol, 1)    # Buy call
        
        self.condor_opened = True

---
## 7. Exercise et Assignment (10 min)

### Options europeennes vs americaines

| Type | Exercise | Utilisation |
|------|----------|-------------|
| **Americaine** | A tout moment avant expiration | Actions US (SPY, AAPL, etc.) |
| **Europeenne** | Uniquement a l'expiration | Indices (SPX), certaines options |

**Note** : Les options sur ETF comme SPY sont **americaines**, donc peuvent etre exercees/assignees a tout moment.

### Exercise manuel

Pour exercer une option que vous detenez :

```python
# Exercer un call (acheter 100 actions au strike)
self.ExerciseOption(call_symbol, 1)  # 1 contrat

# Exercer un put (vendre 100 actions au strike)
self.ExerciseOption(put_symbol, 1)
```

### Assignment automatique

Si vous avez **vendu** une option :

- **Assignment possible** : A tout moment (options americaines) si ITM
- **Assignment garanti** : A l'expiration si ITM

Consequences :
- **Call assigne** : Vous devez vendre 100 actions au strike
- **Put assigne** : Vous devez acheter 100 actions au strike

### Evenement OnAssignmentOrderEvent

QuantConnect declenche cet evenement lors d'un assignment :

In [ ]:
# Gestion des assignments
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class AssignmentHandling(QCAlgorithm):
    """
    Exemple de gestion des assignments d'options.
    """
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetCash(100000)
        
        self.equity = self.AddEquity("SPY", Resolution.Minute)
        self.option = self.AddOption("SPY", Resolution.Minute)
    
    def OnAssignmentOrderEvent(self, assignmentEvent):
        """
        Appele lorsqu'une option vendue est assignee.
        """
        self.Log(f"\n!!! ASSIGNMENT !!!")
        self.Log(f"Symbol: {assignmentEvent.Symbol}")
        self.Log(f"Quantity: {assignmentEvent.FillQuantity}")
        self.Log(f"Fill Price: ${assignmentEvent.FillPrice:.2f}")
        
        # Actions potentielles apres assignment:
        # 1. Laisser la position (si covered call, vous avez deja les actions)
        # 2. Racheter les actions (si call nu assigne)
        # 3. Vendre les actions recues (si put assigne)
    
    def OnOrderEvent(self, orderEvent):
        """
        Appele pour tout evenement d'ordre (fill, cancel, etc.)
        """
        if orderEvent.Status == OrderStatus.Filled:
            self.Debug(f"Order filled: {orderEvent.Symbol}, " +
                       f"Qty: {orderEvent.FillQuantity}, " +
                       f"Price: ${orderEvent.FillPrice:.2f}")

### Early assignment - Quand se mefier

L'assignment anticipe (avant expiration) est plus probable dans ces cas :

| Situation | Risque d'assignment |
|-----------|---------------------|
| Call ITM proche du dividende | **Eleve** - L'acheteur veut le dividende |
| Put profondement ITM | **Moyen** - L'acheteur veut le cash |
| Proche de l'expiration | **Eleve** - Plus de valeur temporelle |
| Option sans valeur temporelle | **Eleve** - Pas de raison d'attendre |

**Conseil** : Pour eviter l'assignment anticipe, fermez vos positions short quelques jours avant l'expiration si elles sont ITM.

---
## 8. Strategie Complete : Covered Call sur SPY (20 min)

### Objectif

Implementer une strategie **Covered Call systematique** qui :

1. Achete 100 actions SPY
2. Vend un call OTM mensuel (delta ~0.30)
3. Roll le call avant expiration (si non assigne)
4. Track le premium total collecte

### Parametres de la strategie

| Parametre | Valeur | Justification |
|-----------|--------|---------------|
| Delta cible | 0.25-0.35 | Balance entre premium et probabilite d'assignment |
| Expiration | 30-45 jours | Bon decay theta, pas trop de risque gamma |
| Roll | 7 jours avant expiry | Eviter le gamma risk de fin de mois |
| Capital | 50,000$ | Suffisant pour 100 SPY + marge |

### Criteres de selection du call

1. **Type** : Call
2. **Strike** : OTM (strike > prix actuel)
3. **Delta** : Entre 0.25 et 0.35
4. **Expiration** : 30-45 jours
5. **Open Interest** : > 100 (liquidite)

In [ ]:
# Strategie Covered Call complete avec roll
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *
from datetime import timedelta

class CoveredCallWithRoll(QCAlgorithm):
    """
    Strategie Covered Call systematique sur SPY avec roll automatique.
    
    Features:
    - Achat initial de 100 SPY
    - Vente de calls OTM (delta 0.30)
    - Roll 7 jours avant expiration
    - Tracking du premium collecte
    """
    
    def Initialize(self):
        # Configuration
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(55000)  # Suffisant pour 100 SPY
        
        # Ajouter l'equity
        self.equity = self.AddEquity("SPY", Resolution.Minute)
        self.spy = self.equity.Symbol
        
        # Ajouter les options
        self.option = self.AddOption("SPY", Resolution.Minute)
        self.option_symbol = self.option.Symbol
        
        # Filtre: calls OTM, 30-50 jours
        self.option.SetFilter(
            minStrike=0,
            maxStrike=15,
            minExpiry=timedelta(days=25),
            maxExpiry=timedelta(days=50)
        )
        
        # Parametres de la strategie
        self.target_delta = 0.30
        self.delta_tolerance = 0.05
        self.days_to_roll = 7  # Roll 7 jours avant expiration
        
        # Tracking
        self.current_call = None
        self.current_call_expiry = None
        self.total_premium = 0
        self.trades_count = 0
        
        # Scheduled function pour check quotidien
        self.Schedule.On(
            self.DateRules.EveryDay("SPY"),
            self.TimeRules.AfterMarketOpen("SPY", 30),
            self.ManageCoveredCall
        )
        
        self.Log("=== Covered Call Strategy Initialized ===")
        self.Log(f"Target Delta: {self.target_delta}")
        self.Log(f"Roll {self.days_to_roll} days before expiry")
    
    def ManageCoveredCall(self):
        """
        Fonction quotidienne pour gerer le covered call.
        """
        # Etape 1: Acheter 100 SPY si necessaire
        if not self.Portfolio[self.spy].Invested:
            quantity = 100
            self.MarketOrder(self.spy, quantity)
            self.Log(f"[INIT] Bought {quantity} SPY shares")
            return
        
        # Etape 2: Verifier si on doit roller
        if self.current_call_expiry is not None:
            days_to_expiry = (self.current_call_expiry - self.Time.date()).days
            
            if days_to_expiry <= self.days_to_roll:
                self.RollCall()
                return
        
        # Etape 3: Vendre un call si on n'en a pas
        if self.current_call is None:
            self.SellCall()
    
    def SellCall(self):
        """
        Vendre un call OTM avec le delta cible.
        """
        # Obtenir la chaine d'options
        chain = self.CurrentSlice.OptionChains.get(self.option_symbol)
        if chain is None:
            return
        
        underlying_price = chain.Underlying.Price
        
        # Filtrer les calls OTM
        calls = [c for c in chain 
                 if c.Right == OptionRight.Call
                 and c.Strike > underlying_price
                 and c.OpenInterest > 100]  # Liquidite
        
        if not calls:
            self.Debug("No suitable calls found")
            return
        
        # Selectionner par delta
        suitable_calls = [c for c in calls 
                          if abs(c.Greeks.Delta - self.target_delta) <= self.delta_tolerance]
        
        if not suitable_calls:
            # Si pas de call avec bon delta, prendre le plus proche
            selected = min(calls, key=lambda x: abs(x.Greeks.Delta - self.target_delta))
        else:
            # Prendre celui avec le meilleur premium parmi les candidats
            selected = max(suitable_calls, key=lambda x: x.BidPrice)
        
        # Vendre le call
        self.MarketOrder(selected.Symbol, -1)
        
        # Mettre a jour le tracking
        self.current_call = selected.Symbol
        self.current_call_expiry = selected.Expiry.date()
        
        premium = selected.BidPrice * 100
        self.total_premium += premium
        self.trades_count += 1
        
        self.Log(f"[SELL] Call Strike={selected.Strike}, " +
                 f"Expiry={selected.Expiry.date()}, " +
                 f"Delta={selected.Greeks.Delta:.3f}, " +
                 f"Premium=${premium:.2f}")
    
    def RollCall(self):
        """
        Fermer le call actuel et en vendre un nouveau.
        """
        self.Log(f"[ROLL] Rolling call before expiration")
        
        # Fermer le call actuel (acheter pour couvrir)
        if self.current_call is not None:
            self.MarketOrder(self.current_call, 1)  # Buy to close
            self.current_call = None
            self.current_call_expiry = None
        
        # Vendre un nouveau call
        self.SellCall()
    
    def OnAssignmentOrderEvent(self, assignmentEvent):
        """
        Gerer l'assignment du call.
        """
        self.Log(f"[ASSIGNMENT] Call assigned at {assignmentEvent.FillPrice}")
        self.Log(f"Actions vendues, rachat necessaire")
        
        # Reset le tracking
        self.current_call = None
        self.current_call_expiry = None
        
        # Note: A la prochaine iteration, on rachete 100 SPY
    
    def OnEndOfAlgorithm(self):
        """
        Resume de la performance.
        """
        initial_value = self.StartingCash
        final_value = self.Portfolio.TotalPortfolioValue
        total_return = (final_value - initial_value) / initial_value * 100
        
        self.Log(f"\n{'='*60}")
        self.Log(f"COVERED CALL STRATEGY - FINAL REPORT")
        self.Log(f"{'='*60}")
        self.Log(f"Period: {self.StartDate.date()} to {self.EndDate.date()}")
        self.Log(f"Initial Capital: ${initial_value:,.2f}")
        self.Log(f"Final Value: ${final_value:,.2f}")
        self.Log(f"Total Return: {total_return:.2f}%")
        self.Log(f"")
        self.Log(f"PREMIUM COLLECTION:")
        self.Log(f"  Total Premium Collected: ${self.total_premium:,.2f}")
        self.Log(f"  Number of Trades: {self.trades_count}")
        if self.trades_count > 0:
            self.Log(f"  Average Premium/Trade: ${self.total_premium/self.trades_count:.2f}")
        self.Log(f"{'='*60}")

### Points cles de l'implementation

1. **Schedule.On** : Execution quotidienne de `ManageCoveredCall` 30 minutes apres l'ouverture du marche

2. **Selection du call** :
   - Filtre par delta (0.25-0.35)
   - Verifie la liquidite (Open Interest > 100)
   - Prend le meilleur premium parmi les candidats

3. **Roll automatique** :
   - Ferme la position 7 jours avant expiration
   - Evite le risque gamma eleve de fin de mois
   - Ouvre immediatement une nouvelle position

4. **Gestion de l'assignment** :
   - Log l'evenement
   - Reset le tracking
   - Rachete 100 SPY a la prochaine iteration

### Metriques a analyser

| Metrique | Description | Valeur cible |
|----------|-------------|---------------|
| Total Premium | Somme des premiums collectes | > 5% du capital/an |
| Win Rate | % des trades expires worthless | > 70% |
| Sharpe Ratio | Rendement ajuste au risque | > 1.0 |
| Max Drawdown | Perte maximale | < 20% |

---
## 9. Conclusion

### Recapitulatif

Dans ce notebook, nous avons couvert :

#### 1. Fondamentaux des Options
- **Call** : Droit d'acheter
- **Put** : Droit de vendre
- **ITM/ATM/OTM** : Relation strike/prix

#### 2. Configuration dans QuantConnect
- `AddOption("SPY", Resolution.Minute)`
- `SetFilter(-5, 5, timedelta(30), timedelta(60))`

#### 3. Navigation dans l'OptionChain
- Acces via `slice.OptionChains[symbol]`
- Selection par Strike, Expiry, Delta

#### 4. Greeks
- **Delta** : Sensibilite au prix
- **Theta** : Decay temporel
- **Vega** : Sensibilite a la volatilite

#### 5. Strategies
| Strategie | Objectif | Complexite |
|-----------|----------|------------|
| Covered Call | Revenu | Simple |
| Protective Put | Protection | Simple |
| Bull Call Spread | Hausse limitee | Moyen |
| Iron Condor | Marche lateral | Avance |

### Points importants a retenir

1. **Resolution.Minute obligatoire** pour les options
2. **1 contrat = 100 actions** - Attention au sizing
3. **Toujours verifier la liquidite** (Open Interest, Bid-Ask spread)
4. **Greeks pour le risk management** - Surtout Delta et Theta
5. **Assignment possible a tout moment** pour options americaines

### Prochaines etapes

Pour approfondir :

1. **Tester les strategies** sur QuantConnect avec differents parametres
2. **Analyser les resultats** : Sharpe, Drawdown, Win Rate
3. **Explorer les strategies avancees** : Butterfly, Calendar Spread, Straddle
4. **Combiner avec analyse technique** : RSI, MACD pour timing d'entree

### Ressources supplementaires

**Documentation QuantConnect** :
- [Options Trading Documentation](https://www.quantconnect.com/docs/v2/writing-algorithms/securities/asset-classes/equity-options)
- [Greeks Reference](https://www.quantconnect.com/docs/v2/writing-algorithms/securities/asset-classes/equity-options/greeks)

**Livres recommandes** :
- *Options, Futures, and Other Derivatives* - John C. Hull
- *The Options Playbook* - Brian Overby

---

**Felicitations !** Vous maitrisez maintenant les bases du trading d'options sur QuantConnect. Les options offrent une flexibilite enorme pour la gestion de portefeuille, la generation de revenus, et la speculation. Utilisez ces connaissances avec prudence et commencez toujours par des positions petites.

**Prochain notebook** : QC-Py-07 - Futures et Commodities